<a href="https://colab.research.google.com/github/PurvaChiniya/Aspect-based-sentiment-analysis/blob/main/create_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Install dependencies
!pip uninstall -y tensorflow
!pip install transformers
import json
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import transformers
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score
from transformers import BertModel, BertTokenizer

import logging
logging.basicConfig(level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore")


Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
     |████████████████████████████████| 3.4 MB 8.2 MB/s 
     |████████████████████████████████| 61 kB 503 kB/s 
     |████████████████████████████████| 596 kB 56.3 MB/s 
     |████████████████████████████████| 895 kB 51.8 MB/s 
     |████████████████████████████████| 3.3 MB 36.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [12]:
# declare parameters
max_length = 160
batch_size = 32
locations = ['LOCATION1', 'LOCATION2']
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']
labels_to_sentiment = {
    0: 'Positive',
    1: 'Negative',
    2: 'None'
}
bert_model = 'bert-base-uncased'
with open('/content/drive/MyDrive/sentihood/sentihood-test.json', 'r') as f:
  testing_set = json.load(f)[:400]

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")


Device: cpu


In [13]:

import warnings
warnings.filterwarnings("ignore")
class Model(nn.Module):
  # same as train 
  def __init__(self, BERT_MODEL):
    super(Model, self).__init__()
    self.bert = BertModel.from_pretrained(BERT_MODEL)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, 3) # Number of output classes = 3

  def forward(self, ids, mask, token_type_ids):
    last_hidden_state, pooled_output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
    output = self.drop(pooled_output)
    return self.out(output)

model = torch.load('/content/drive/MyDrive/sentihood/1.bin')


In [14]:
for each_example in tqdm(testing_set, ncols=80):
  id = each_example['id']
  text = each_example['text'].strip()

  each_example['model_pred'] = []

  count_location = 1
  for location in locations:
    if location in text:
      # If "location" is present in the text, then utilize the trained model
      # to predict the aspects and their corresponding sentiment of the text.

      text = text.replace(location, 'location - ' + str(count_location))
      
      for aspect in aspects:
        auxiliary_sentence = f'location - {str(count_location)} - {aspect}'
        combined_text = text + ' ' + auxiliary_sentence
        
        inputs = tokenizer.encode_plus(
            combined_text,
            add_special_tokens = True,
            max_length = max_length,
            pad_to_max_length = True
        )
        ids = torch.tensor(inputs["input_ids"], dtype=torch.long).unsqueeze(0)
        mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).unsqueeze(0)
        token_type_ids = torch.tensor(inputs["token_type_ids"], dtype=torch.long).unsqueeze(0)

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        _, predicted = torch.max(outputs, 1)

        predicted = predicted.detach().cpu().numpy()

         # If predicted sentiment is not None, then add it to the preds.jsonl.
         
        if predicted[0] != 2:
          result = {
              "sentiment": labels_to_sentiment_dict[predicted[0]],
              "aspect": aspect,
              "target_entity": location
          }
          each_example['model_pred'].append(result)
      
    count_location += 1

100%|███████████████████████████████████████| 400/400 [1:00:26<00:00,  9.07s/it]


In [15]:
with open('/content/pred.jsonl', mode='w', encoding='utf-8') as fp:
  for each in testing_set:
    json_record = json.dumps(each, ensure_ascii=False)
    fp.write(json_record + '\n')